## Power of MPO
For $H = J\sum Z\otimes Z + \sum L$, we have MPO given by
$$
\begin{pmatrix}
1\\
Z\\
H\\
\end{pmatrix}'\leftarrow
\begin{bmatrix}
1& &\\
Z& &\\
L& JZ& 1\\
\end{bmatrix}
\begin{pmatrix}
1\\
Z\\
H\\
\end{pmatrix}$$

For $H^2$, its MPO is given by
$$
\begin{pmatrix}
1\\
Z\\
H\\
Z^2\\
(HZ)\\
H^2
\end{pmatrix}'\leftarrow
\begin{bmatrix}
1& &&&&\\
Z& & & & &\\
L& JZ& 1& & & \\
Z^2& & & & & \\
(ZL)& JZ^2& Z& & & \\
L^2& 2J(ZL)& 2L& J^2Z^2& 2JZ& 1
\end{bmatrix}
\begin{pmatrix}
1\\
Z\\
H\\
Z^2\\
(HZ)\\
H^2
\end{pmatrix}
$$

$(JZ_1Z_2+L_1+L_2)^2=J^2Z_1^2Z_2^2+L_1^2+L_2^2+2L_1L_2+2J(LZ)_1Z_2+2J(LZ)_2Z_1$

# Generalization
If some Hamiltonian $H$ has $k+1$ MPO bases $(1, O_1, O_2,\ldots , O_{k-1}, H)$, bases of its power $H^n$ can be reduced to $$\binom{k+n}{n}=\frac{(k+n)!}{k! n!}$$

In [45]:
import numpy as np
from functools import reduce
def dense_form(MPO):
    start = np.zeros(MPO[0].shape[0])
    start[0] = 1
    def add(l, m):
        L = len(l)
        return [np.sum([np.kron(l[j], m[j, i]) for j in range(i+1)], axis=0) for i in range(L)]
    return reduce(add, [start, *MPO])[-1]

In [88]:
from DMRG.spin import sigma
# Pauli Matrices
I, X, Y, Z = sigma
# Zero Matrix
O = np.zeros([2,2])
def MPO_TL(J=1, g=1, h=1):
    '''$H=-\sum J Z_iZ_{i+1}+\sum (gX_i+hZ_i)$
    NOTE The matrix of operators is upper-triangle after transosition
    '''
    L = g*X+h*Z # local terms
    LZ = (L@Z+Z@L)/2
    mpo = np.array([
        [I,     O,      O,      O,      O,      O],
        [Z,     O,      O,      O,      O,      O],
        [L,    -J*Z,    I,      O,      O,      O],
        [Z@Z,   O,      O,      O,      O,      O],
        [LZ,   -J*Z@Z,  Z,      O,      O,      O],
        [L@L,  -2*J*LZ, 2*L,  J*J*Z@Z,  -2*J*Z,  I],
        ]).transpose([1,0,2,3])
    return mpo[:3, :3], mpo

In [89]:
from DMRG.Ising import Hamilton_TL

In [90]:
H, H2 = MPO_TL()

In [91]:
H2_1@H2_1

array([[ 10.,   2.,   4., ...,   0.,   0.,   0.],
       [  2.,  10.,   2., ...,   0.,   0.,   0.],
       [  4.,   2.,  18., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ..., 130.,   2., -28.],
       [  0.,   0.,   0., ...,   2., 178., -30.],
       [  0.,   0.,   0., ..., -28., -30., 298.]])

In [92]:
n = 2
H_ = dense_form([H]*n)
H2_ = dense_form([H2]*n)
H2_-H_@H_

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [80]:
H_2 = Hamilton_TL(9, 1, 1, 1)['H']